<a href="https://colab.research.google.com/github/ibacaraujo/pysyft-learning/blob/master/Part_03_Advanced_Remote_Execution_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 03. Advanced Remote Execution Tools

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.7MB 2.9MB/s 
     |████████████████████████████████| 276kB 28.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=44229 sha256=8c5f5ae63d7fbdf935cf8cdff81cbbdda5e6e013efdc505251b62e5817dad3fe
  Stored in directory: /root/.cache/pip/wheels/e4/76/4d/a95b8dd7b452b69e8ed4f68b69e1b55e12c9c9624dd962b191
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 31660 (delta 0), reused 1 (delta 0), pack-reused 31659
Receiving objects: 100% (31660/31660), 33.65 MiB | 20.91 MiB/s, done.
Resolving deltas: 100% (21338/21338), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |███████████████████████

In [3]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0.so'


## 3.1. Pointers to Pointers

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [5]:
# this is a local tensor
x = torch.tensor([1, 2, 3, 4])
x

tensor([1, 2, 3, 4])

In [6]:
# this sends the local tensor to Bob
x_ptr = x.send(bob)

# this is now a pointer
x_ptr

(Wrapper)>[PointerTensor | me:58234562885 -> bob:92293954287]

In [7]:
# now we can SEND THE POINTER to Alice!
pointer_to_x_ptr = x_ptr.send(alice)

pointer_to_x_ptr

(Wrapper)>[PointerTensor | me:32497951427 -> alice:58234562885]

In [8]:
# As you can see, Bob still has the actual data
# Data is always stored in a LocalTensor type
bob._objects

{92293954287: tensor([1, 2, 3, 4])}

In [10]:
# Alice, on the other hand, has x_ptr!
alice._objects

{58234562885: (Wrapper)>[PointerTensor | alice:58234562885 -> bob:92293954287]}

In [0]:
# and we can use .get() to get x_ptr back from Alice
x_ptr = pointer_to_x_ptr.get()

In [12]:
x_ptr

(Wrapper)>[PointerTensor | me:58234562885 -> bob:92293954287]

In [14]:
# and then we can use x_ptr to get x back from Bob!
x = x_ptr.get()
x

tensor([1, 2, 3, 4])

Like normal pointers, we can perform arbitrary PyTorch operations across these tensors

In [15]:
bob._objects

{}

In [16]:
alice._objects

{}

In [0]:
p2p2x = torch.tensor([1, 2, 3, 4, 5]).send(bob).send(alice)

y = p2p2x + p2p2x

In [18]:
y

(Wrapper)>[PointerTensor | me:67401355910 -> alice:80970247146]

In [19]:
bob._objects

{27555915638: tensor([1, 2, 3, 4, 5]),
 57996403960: tensor([ 2,  4,  6,  8, 10])}

In [20]:
alice._objects

{39267930835: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960],
 47593992032: (Wrapper)>[PointerTensor | alice:47593992032 -> bob:27555915638],
 80970247146: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960]}

In [21]:
y.get().get()

tensor([ 2,  4,  6,  8, 10])

In [22]:
bob._objects

{27555915638: tensor([1, 2, 3, 4, 5])}

In [23]:
alice._objects

{39267930835: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960],
 47593992032: (Wrapper)>[PointerTensor | alice:47593992032 -> bob:27555915638]}

In [24]:
p2p2x.get().get()

tensor([1, 2, 3, 4, 5])

In [25]:
bob._objects

{}

In [26]:
alice._objects

{39267930835: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960]}

## 3.2. Pointer Chain Operations

In [0]:
# x is now a pointer to the data which lives on Bob's machine
x = torch.tensor([1, 2, 3, 4, 5]).send(bob)

In [28]:
print('bob:', bob._objects)
print('alice:', alice._objects)

bob: {90741221040: tensor([1, 2, 3, 4, 5])}
alice: {39267930835: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960]}


In [29]:
x

(Wrapper)>[PointerTensor | me:87484071636 -> bob:90741221040]

In [0]:
x = x.move(alice)

In [31]:
print('bob:', bob._objects)
print('alice:', alice._objects)

bob: {}
alice: {39267930835: (Wrapper)>[PointerTensor | alice:80970247146 -> bob:57996403960], 87484071636: tensor([1, 2, 3, 4, 5])}


In [32]:
x

(Wrapper)>[PointerTensor | me:29348085545 -> alice:87484071636]